In [190]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [191]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SETS = tuple(np.array([random() < .4 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

In [192]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [193]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable" # sometimes an error may occur

In [194]:
def search(frontier):
    current_state = frontier.get()

    # to count the number of sets put in the frontier
    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(new_state)
        current_state = frontier.get()

    print(f"Solved in {counter} steps")
    return current_state

Breadth First Search and Depth First Search

In [195]:
fifo = SimpleQueue()
lifo = LifoQueue()

fifo.put(State(set(), set(range(NUM_SETS))))
lifo.put(State(set(), set(range(NUM_SETS))))

breadth_solution_state = search(fifo)
depth_solution_state = search(lifo)

Solved in 32 steps
Solved in 6 steps


In [196]:
print(
    f"Solution with a Breadth Search:\n{breadth_solution_state}\nSolution with a Depth Search:\n{depth_solution_state}"
)

Solution with a Breadth Search:
State(taken={2, 4}, not_taken={0, 1, 3, 5, 6, 7, 8, 9})
Solution with a Depth Search:
State(taken={4, 5, 6, 7, 8, 9}, not_taken={0, 1, 2, 3})


Dijkstra Search

In [197]:
def cost_function(taken):
    return sum([np.sum(SETS[i]) for i in taken])

In [198]:
def search_dijkstra(frontier):
    current_state = frontier.get()

    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((cost_function(new_state.taken), new_state))
        current_state = frontier.get()[1]

    print(f"Solved in {counter} steps")
    return current_state

In [199]:
frontier = PriorityQueue()
frontier.put(State(set(), set(range(NUM_SETS))))
dijkstra_solution_state = search_dijkstra(frontier)

Solved in 1362 steps


In [200]:
dijkstra_solution_state

State(taken={2, 6}, not_taken={0, 1, 3, 4, 5, 7, 8, 9})

Gready Search

In [201]:
def distance(state):
    uncovered = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or, 
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return uncovered

def greedy_best_searh(frontier):
    current_state = frontier.get()

    steps = 0
    while not goal_check(current_state):
        steps += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((distance(new_state), new_state))
        _, current_state = frontier.get()
    print(f"Solved in {steps} steps and with {len(current_state.taken)} tiles")
    return current_state

In [202]:
frontier = PriorityQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

greedy_solution_search = greedy_best_searh(frontier)
print([SETS[i] for i in greedy_solution_search.taken])

Solved in 2 steps and with 2 tiles
[array([ True,  True, False,  True,  True]), array([False, False,  True, False,  True])]


A star

In [203]:
def is_special(s):
    return sum(s) / PROBLEM_SIZE >= 0.6

def g_cost(state):
    return len(state.taken)

def distance(state):
    uncovered = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or, 
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

    for i in state.not_taken:
        if is_special(SETS[i]):
            return uncovered - 0.5
    return uncovered

# A* approach
def astar_search():
    frontier = PriorityQueue()
    start_state = State(set(), set(range(NUM_SETS)))
    frontier.put((0 + distance(start_state), start_state))
    
    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        sorted_actions = sorted(list(current_state.not_taken), key=lambda x: -sum(SETS[x]))
        for action in sorted_actions:
            new_taken = set(current_state.taken ^ {action})
            new_not_taken = set(current_state.not_taken ^ {action})
            new_state = State(new_taken, new_not_taken)
            frontier.put((g_cost(new_state) + distance(new_state), new_state))
        counter += 1
        _, current_state = frontier.get()
    return counter, current_state
counter, current_state = astar_search()
print(f"Solved using A* in {counter:,} steps with {len(current_state.taken)} sets")

Solved using A* in 2 steps with 2 sets


New A star

In [204]:
def g_cost(state):
    return len(state.taken)

def distance(state):
    uncovered = PROBLEM_SIZE - sum(
        reduce(
            np.logical_or, 
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return uncovered

def A_search(frontier):
    current_state = frontier.get()

    counter = 0
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(
                (
                    g_cost(new_state) + distance(new_state),
                    new_state,
                )
            )
        current_state = frontier.get()[1]

    print(f"Solved in {counter} steps")
    return current_state

In [205]:
frontier = PriorityQueue()

frontier.put(State(set(), set(range(NUM_SETS))))

A_star_solution_search = A_search(frontier)

Solved in 2 steps
